In [1]:
import yfinance as yf

# Fetch the GME stock data
gme = yf.Ticker("GME")

# Get stock information
gme_info = gme.info
print(gme_info)

# Get historical market data (default period is '1mo')
gme_hist = gme.history(period="1mo")
print(gme_hist)

# Get actions (dividends, splits)
gme_actions = gme.actions
print(gme_actions)

# Get recent stock price
gme_price = gme.history(period="1d")['Close'][0]
print(f"Most recent GME stock price: {gme_price}")


{'address1': '625 Westport Parkway', 'city': 'Grapevine', 'state': 'TX', 'zip': '76051', 'country': 'United States', 'phone': '817 424 2000', 'website': 'https://www.gamestop.com', 'industry': 'Specialty Retail', 'industryKey': 'specialty-retail', 'industryDisp': 'Specialty Retail', 'sector': 'Consumer Cyclical', 'sectorKey': 'consumer-cyclical', 'sectorDisp': 'Consumer Cyclical', 'longBusinessSummary': 'GameStop Corp., a specialty retailer, provides games and entertainment products through its stores and ecommerce platforms in the United States, Canada, Australia, and Europe. The company sells new and pre-owned gaming platforms; accessories, such as controllers, gaming headsets, and virtual reality products; new and pre-owned gaming software; and in-game digital currency, digital downloadable content, and full-game downloads. It sells collectibles comprising apparel, toys, trading cards, gadgets, and other retail products for pop culture and technology enthusiasts, as well as engages 

/var/folders/kz/_jxr42t969j3x9sdgjxvqjsc0000gn/T/ipykernel_95201/1514107255.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gme_price = gme.history(period="1d")['Close'][0]
